<a href="https://colab.research.google.com/github/dewhitee/ml/blob/main/ml_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import numpy as np
import sklearn as sk
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [74]:
# Loading dataset
heart_failures = read_csv('heart_failure_clinical_records_dataset.csv')

# Dataset shape
total_observations, total_variables = heart_failures.shape
print("Number of observations: ", total_observations)
print("Number of variables: ", total_variables)
total_x_variables = total_variables - 1

heart_failures

Number of observations:  299
Number of variables:  13


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [101]:
# 1. Split dataset into training and testing sets
train, test = train_test_split(heart_failures, shuffle=True,
                               train_size=0.75, test_size=0.25)

# Splitting data for training and validation
x_train = train.values[:, :total_variables - 1]
y_train = train.values[:, total_variables - 1]

x_test = test.values[:, :total_variables - 1]
y_test = test.values[:, total_variables - 1]

print(x_train[0])
print(len(x_train))

[7.90e+01 1.00e+00 5.50e+01 0.00e+00 5.00e+01 1.00e+00 1.72e+05 1.80e+00
 1.33e+02 1.00e+00 0.00e+00 7.80e+01]
224
y_train[0] =  0.0
x_train[0] =  [7.90e+01 1.00e+00 5.50e+01 0.00e+00 5.00e+01 1.00e+00 1.72e+05 1.80e+00
 1.33e+02 1.00e+00 0.00e+00 7.80e+01]


In [104]:
print('y_train[0] = ', y_train[0])
print('x_train[0] = ', x_train[0])
print('x_train[1] = ', x_train[1])

y_train[0] =  0.0
x_train[0] =  [7.90e+01 1.00e+00 5.50e+01 0.00e+00 5.00e+01 1.00e+00 1.72e+05 1.80e+00
 1.33e+02 1.00e+00 0.00e+00 7.80e+01]
x_train[1] =  [5.10e+01 0.00e+00 7.80e+01 0.00e+00 5.00e+01 0.00e+00 4.06e+05 7.00e-01
 1.40e+02 1.00e+00 0.00e+00 7.90e+01]


In [135]:
# 2. Single-layer perceptron algorithm
def make_prediction(observation, weight_coefficients):
    # Y_in = w0 + w1*x1 + w2*x2 + ... wn*xn
    y_in = weight_coefficients[0] # W0 or activation
    # iterating over each attribute of current observation
    for attribute_index in range(len(observation)):
        # + W1 * X1 + ... + Wn * Xn
        #print('y_in += ', observation[attribute_index], "*", weight_coefficients[attribute_index + 1])
        y_in += observation[attribute_index] * weight_coefficients[attribute_index + 1]

    # Heaviside (calculating out_y - prediction)
    #print('y_in = ', y_in)
    return 0 if y_in <= 0 else 1

def correct_weight_coefficients(weight_coefficients, learning_rate,
                                observation, current_error):
    # Correction of weight coefficients vector
    #weight_coefficients[0] = -learning_rate * current_error
    weight_coefficients[0] -= learning_rate * current_error
    for i in range(1, total_x_variables):
        #weight_coefficients[i] = -learning_rate * current_error * observation[i]
        weight_coefficients[i] -= learning_rate * current_error * observation[i]

def get_trained_weights(x_train, y_train, learning_rate = 0.1, # our alpha - value in range (0, 1)
                    initial_value = 0.5,
                    epochs = 10): 
    # init weight coefficients with initial_value
    # We are adding 1 to total variables because we need to add
    # bias weight coefficient - w0
    weight_coefficients = [initial_value] * (total_x_variables + 1)
    #print(f'Init weight len: {len(weight_coefficients)}, coefficients: {weight_coefficients}')

    for i in range(epochs):
        print(' --- Epoch:', i, '---')
        y_results = []

        # Sum of errors accumulated for the current epoch
        error_sum = 0.0

        # Propagation
        for index, observation in enumerate(x_train):
            # Get current prediction y (activation) value
            prediction = make_prediction(observation, weight_coefficients)
            #print('prediction = ', prediction)
            y_results.append(prediction)

            # Calculating current error value (Y - t)
            current_error = prediction - y_train[index]

            # Updating total current-epoch errors sum
            error_sum += current_error

            #print('weight_coefficients len = ', len(weight_coefficients),
            #      ', y_results len = ', len(y_results), ', learning_rate = ', learning_rate)
            correct_weight_coefficients(weight_coefficients, learning_rate,
                                        observation, current_error)
            
            #print("iteration = ", index, ", error_sum =", error_sum, ", weights: ", weight_coefficients)

        print('y_results length: ', len(y_results), ', error sum: ', error_sum)
        print('y_results: ', y_results[:10], '...', y_results[-10:])
        print('y_train: ', y_train[:10], '...', y_train[-10:])
        print('accuracy: ', accuracy_score(y_train, y_results))

    return weight_coefficients

print('y_train[0] = ', y_train[0])
print('x_train[0] = ', x_train[0])

# Train (fit) perceptron model
model_weights = get_trained_weights(x_train, y_train, learning_rate=0.1,
                                   initial_value=0.5,
                                   epochs=1)

predictions = []
for observation in x_test:
    predicted = make_prediction(observation, model_weights)
    predictions.append(predicted)



y_train[0] =  0.0
x_train[0] =  [7.90e+01 1.00e+00 5.50e+01 0.00e+00 5.00e+01 1.00e+00 1.72e+05 1.80e+00
 1.33e+02 1.00e+00 0.00e+00 7.80e+01]
 --- Epoch: 0 ---
y_results length:  224 , error sum:  25.0
y_results:  [1, 1, 0, 1, 1, 1, 0, 1, 1, 0] ... [1, 0, 0, 0, 0, 0, 0, 1, 1, 1]
y_train:  [0. 0. 0. 1. 1. 0. 0. 0. 1. 0.] ... [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
accuracy:  0.5401785714285714


In [64]:
print('y_train[:10]', y_train[:10])

y_train[:10] [0. 1. 0. 1. 1. 0. 0. 0. 0. 0.]


In [125]:
# Compare with keras perceptron
from sklearn.linear_model import Perceptron

slp = Perceptron(max_iter=2000, verbose=1)
slp.fit(x_train, y_train)
print('Predicted: ', slp.predict(x_test))

-- Epoch 1
Norm: 728153.47, NNZs: 12, Bias: 0.000000, T: 224, Avg. loss: 17773151676.541416
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 241143.14, NNZs: 12, Bias: 5.000000, T: 448, Avg. loss: 17942872414.078350
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 171315.13, NNZs: 12, Bias: 8.000000, T: 672, Avg. loss: 17877807942.349751
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 343419.30, NNZs: 12, Bias: 15.000000, T: 896, Avg. loss: 16276919596.776386
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 348818.15, NNZs: 12, Bias: 13.000000, T: 1120, Avg. loss: 16625258087.412907
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 181525.08, NNZs: 12, Bias: 17.000000, T: 1344, Avg. loss: 15741507095.409922
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 272234.49, NNZs: 12, Bias: 20.000000, T: 1568, Avg. loss: 14684207779.935085
Total training time: 0.01 seconds.
-- Epoch 8
Norm: 147638.51, NNZs: 12, Bias: 22.000000, T: 1792, Avg. loss: 16907063170.022823
Total trai

In [123]:
print(x_train[:, 0])

[79.    51.    72.    94.    70.    63.    60.    40.    58.    63.
 65.    55.    45.    60.    75.    69.    65.    65.    70.    42.
 45.    60.    55.    50.    60.    80.    60.    45.    69.    75.
 50.    50.    55.    45.    52.    43.    70.    65.    40.    49.
 55.    62.    60.667 87.    60.    72.    50.    42.    45.    58.
 58.    52.    81.    60.    50.    65.    65.    52.    54.    50.
 53.    78.    58.    75.    77.    60.    55.    57.    55.    65.
 49.    50.    95.    60.    45.    85.    65.    72.    73.    65.
 80.    70.    75.    85.    41.    50.    60.    63.    60.    65.
 72.    60.    50.    78.    90.    60.    63.    63.    85.    60.
 72.    50.    72.    60.    70.    52.    75.    50.    49.    63.
 65.    80.    40.    60.    40.    50.    65.    70.    58.    61.
 70.    80.    70.    50.    61.    60.    53.    42.    62.    86.
 50.    58.    61.    52.    62.    42.    45.    53.    60.    58.
 59.    60.    53.    44.    70.    53.    40.  